# Stress Test Yaw-Rate
This is brief example of using AdaStress to adaptively stress test an external simulation. This is done by sending and receiving data over TCP between AdaStress and the main vehicle simulation.

In [1]:

using Pkg
Pkg.activate(".")

include("transceive_data.jl")

  Activating new project at `~/Repos/simple-vehicle-sim/ST_yaw_rate`


start_server (generic function with 1 method)

In [2]:
using AdaStress
import AdaStress.BlackBox
# using Plots

# Simulation

In [3]:
using Distributions

In [4]:
Base.@kwdef mutable struct simple_vehicle <: AdaStress.BlackBox
	t::Float64 = 0.0
	t_max::Float64 = 10.0
	psi_dot::Float64 = 0.0 # the yaw-rate of the vehicle
	psi_dot_fail::Float64 = 3.0 # radians per second
	dist::Normal = Normal(0.0, 1.0)
	log::Vector{Float64} = []
end


function initialize!(sim::simple_vehicle)
	sim.t = 0.0
	sim.psi_dot = 0.0
	sim.log = [sim.psi_dot]
end

function update!(sim::simple_vehicle)
    # Wait for yaw-rate data
    psi_dot, receive_message()
    sim.t = 
    a = rand(sim.dist)
    sim.x += a
    push!(sim.log, sim.x)
    return logpdf(sim.dist, a) - logpdf(sim.dist, 0.0)
end

update! (generic function with 1 method)

In [5]:
AdaStress.reset!(sim::simple_vehicle) = initialize!(sim)
AdaStress.step!(sim::simple_vehicle) = update!(sim)
AdaStress.isterminal(sim::simple_vehicle) = sim.t >= sim.t_max
AdaStress.isevent(sim::simple_vehicle) = sim.psi_dot >= sim.psi_dot_fail
AdaStress.distance(sim::simple_vehicle) = max(sim.psi_dot_fail - sim.psi_dot, 0.0)

In [6]:
# 9201 is the port number for getting psi_dot data from the simulation
start_server("0.0.0.0", 9201) 


Server listening on 0.0.0.0:9201...
